In [11]:
# Bring in all dependencies

import pandas as pd
import requests
import numpy as np
import json
import time
from datetime import date
from datetime import timedelta
from pprint import pprint
import sqlalchemy
#from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float
#from config_poly import api_key
import sqlite3
#from splinter import Browser
#from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager
import yfinance as yf

In [12]:
# Establish link to the FINRA OATS list of End of Day List of Stocks
# "http://oatsreportable.finra.org/OATSReportableSecurities-EOD.txt"
stock_list_US = pd.read_csv('nyse_list.csv')

stock_list_US = stock_list_US[['Symbol','Name']]

for i,row in stock_list_US.iterrows():
    if '^' in row['Symbol']:
        stock_list_US.drop(i,inplace=True)
        
# Select 100 random tickers and companies from list of stocks
stock_list_US = stock_list_US.sample(n=200)
stock_list_US
# Creates browser window for scraping
#executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path, headless=False)

,Symbol,Name
3039,WK,Workiva Inc. Class A Common Stock
1134,FMS,Fresenius Medical Care AG Common Stock
1012,EQR,Equity Residential Common Shares of Beneficial...
2113,OG,Onion Global Limited American Depositary Share...
1748,MAN,ManpowerGroup Common Stock
...,...,...
806,DCUE,Dominion Energy Inc. 2019 Series A Corporate U...
3106,YEXT,Yext Inc. Common Stock
2983,VRTV,Veritiv Corporation Common Stock
1308,GPMT,Granite Point Mortgage Trust Inc. Common Stock


In [7]:
# Testing yahoo finance code
apple = yf.Ticker('KTN')

apple.info
# Captures today's date and stores as a string
# today = date.today()
# yesterday = today - timedelta(days = 1) 
# todays_date = today.strftime("%Y-%m-%d")
# yesterdays_date = yesterday.strftime("%Y-%m-%d")
# # yesterdays_date
# apple_last_day = apple.history(period="1mo")
# apple_last_day = apple_last_day.reset_index()
# apple_last_day['Ticker'] = 'AAPL'

{'previousClose': 33.38,
 'regularMarketOpen': 33.38,
 'twoHundredDayAverage': 33.018665,
 'trailingAnnualDividendYield': None,
 'payoutRatio': None,
 'volume24Hr': None,
 'regularMarketDayHigh': 33.84,
 'navPrice': None,
 'averageDailyVolume10Day': 2240,
 'totalAssets': None,
 'regularMarketPreviousClose': 33.38,
 'fiftyDayAverage': 32.786285,
 'trailingAnnualDividendRate': None,
 'open': 33.38,
 'toCurrency': None,
 'averageVolume10days': 2240,
 'expireDate': None,
 'yield': None,
 'algorithm': None,
 'dividendRate': None,
 'exDividendDate': None,
 'beta': None,
 'circulatingSupply': None,
 'startDate': None,
 'regularMarketDayLow': 33.4,
 'priceHint': 2,
 'currency': 'USD',
 'regularMarketVolume': 2226,
 'lastMarket': None,
 'maxSupply': None,
 'openInterest': None,
 'marketCap': None,
 'volumeAllCurrencies': None,
 'strikePrice': None,
 'averageVolume': 2034,
 'priceToSalesTrailing12Months': None,
 'dayLow': 33.4,
 'ask': 0,
 'ytdReturn': None,
 'askSize': 800,
 'volume': 2226,
 'f

In [13]:
df_list = []
error_log = open("errors.log","w")

# Loops through list of companies on stock_list_US
for _,row in stock_list_US.iterrows():
    
    # Selects ticker from current row
    ticker = yf.Ticker(row['Symbol'])

    # Grabs last opening price, closing price, and dividend amount from the previous day
    # Note: this was done for consistency purposes, to make sure data from the same time and
    # date were being used to create the related K-means clustering model
    try:
        # Grabs ticker history for the defined period
        ticker_history = ticker.history(period="5y")
        # Resets index for ticker history dataframe in order to make date a unique column
        ticker_history = ticker_history.reset_index()
        # Grabs sector information for ticker 
        sector = ticker.info['sector']
        
        # Creates columns in history df to maintain label consistency
        ticker_history['Symbol'] = row['Symbol']
        ticker_history['Name'] = row['Name']
        ticker_history['Sector'] = sector
            
        # Append ticker history dataframe to list of dataframes
        df_list.append(ticker_history)

    # Exception if error comes up 
    except:
        # Writes problem ticker to error log file
        error_log.write(f"{row['Symbol']}\n")
        # Prints out error message for current ticker
        print(f"On {row['Symbol']}, error trying to call")

On HGH, error trying to call
- ECC           : No data found, symbol may be delisted
On ECC           , error trying to call
On ENBA, error trying to call
On BSX, error trying to call
On TVC, error trying to call
On ASGI, error trying to call
On AEFC, error trying to call
On BSA, error trying to call
On PBC, error trying to call
On TBC, error trying to call
On AKO/A, error trying to call
On ECCY, error trying to call
On AIZN, error trying to call


In [14]:
df = pd.concat(df_list)
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Name,Sector
0,2016-05-31,13.430000,13.860000,13.300000,13.800000,141100.0,0.000,0.0,WK,Workiva Inc. Class A Common Stock,Technology
1,2016-06-01,13.750000,13.880000,13.420000,13.670000,79400.0,0.000,0.0,WK,Workiva Inc. Class A Common Stock,Technology
2,2016-06-02,13.640000,13.790000,13.530000,13.770000,103200.0,0.000,0.0,WK,Workiva Inc. Class A Common Stock,Technology
3,2016-06-03,13.700000,13.900000,13.350000,13.880000,73500.0,0.000,0.0,WK,Workiva Inc. Class A Common Stock,Technology
4,2016-06-06,13.980000,14.000000,13.830000,13.950000,122600.0,0.000,0.0,WK,Workiva Inc. Class A Common Stock,Technology
...,...,...,...,...,...,...,...,...,...,...,...
1254,2021-05-24,26.584748,26.644022,26.219221,26.377287,370000.0,0.000,0.0,CWEN,Clearway Energy Inc. Class C Common Stock,Utilities
1255,2021-05-25,26.545231,26.594626,26.130307,26.327888,629400.0,0.000,0.0,CWEN,Clearway Energy Inc. Class C Common Stock,Utilities
1256,2021-05-26,26.377285,26.653900,26.150064,26.624264,594500.0,0.000,0.0,CWEN,Clearway Energy Inc. Class C Common Stock,Utilities
1257,2021-05-27,26.723054,27.068824,26.624264,26.890999,1157100.0,0.000,0.0,CWEN,Clearway Energy Inc. Class C Common Stock,Utilities


In [9]:
# Note: Please open errors.log and check if the file was updated

MDLQ,ENBA,SOLN,GPJA,DUKH,SOJE,BEPH,GMTA,GD,BRK/B,FVIV,FCRW,


In [20]:
# Vertically concatenates (join) the dataframes in df_list into
# one big dataframe
error_list = []
error_log_2 = open("errors.log","r")

for line in error_log_2:
    
    ticker = line
    
    if '/' in ticker:
        ticker = ticker.replace('/','-')
    if '\n' in line:
        ticker = ticker.replace('\n','')
    error_list.append(ticker)

error_list

['MDLQ',
 'ENBA',
 'SOLN',
 'GPJA',
 'DUKH',
 'SOJE',
 'BEPH',
 'GMTA',
 'GD',
 'BRK-B',
 'FVIV',
 'FCRW']

In [21]:
# Loops through list of companies on stock_list_US
df_list = []

for row in error_list:
    
    # Selects ticker from current row
    ticker = yf.Ticker(row)

    # Grabs last opening price, closing price, and dividend amount from the previous day
    # Note: this was done for consistency purposes, to make sure data from the same time and
    # date were being used to create the related K-means clustering model
    try:
        # Grabs ticker history for the defined period
        ticker_history = ticker.history(period="5y")
        # Resets index for ticker history dataframe in order to make date a unique column
        ticker_history = ticker_history.reset_index()
        
        # Grabs list of keys in ticker.info
        info_keys = ticker.info.keys()
        
        # Checks if 'sector' is in info_keys
        if 'sector' in info_keys:
            # Grabs sector information for ticker 
            sector = ticker.info['sector']
        # If 'sector' not in info_keys, sector becomes Not Listed
        else:
            sector = 'Not Listed'
        
        # Creates columns in history df to maintain label consistency
        ticker_history['Symbol'] = row
        ticker_history['Name'] = ticker.info['shortName']
        ticker_history['Sector'] = sector
            
        # Append ticker history dataframe to list of dataframes
        df_list.append(ticker_history)

    # Exception if error comes up 
    except:
        # Prints out error message for current ticker
        print(f"On {row['Symbol']}, error trying to call")

In [27]:
second_df = pd.concat(df_list)

real_df = pd.concat([df,second_df])

real_df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol,Name,Sector
0,2016-05-31,13.430000,13.860000,13.30,13.80,141100.0,0.0,0.0,WK,Workiva Inc. Class A Common Stock,Technology
1,2016-06-01,13.750000,13.880000,13.42,13.67,79400.0,0.0,0.0,WK,Workiva Inc. Class A Common Stock,Technology
2,2016-06-02,13.640000,13.790000,13.53,13.77,103200.0,0.0,0.0,WK,Workiva Inc. Class A Common Stock,Technology
3,2016-06-03,13.700000,13.900000,13.35,13.88,73500.0,0.0,0.0,WK,Workiva Inc. Class A Common Stock,Technology
4,2016-06-06,13.980000,14.000000,13.83,13.95,122600.0,0.0,0.0,WK,Workiva Inc. Class A Common Stock,Technology
...,...,...,...,...,...,...,...,...,...,...,...
659,2021-05-24,25.670000,25.670000,25.67,25.67,0.0,0.0,0.0,FCRW,First Eagle Alternative Capital,Not Listed
660,2021-05-25,25.629999,25.629999,25.57,25.57,1200.0,0.0,0.0,FCRW,First Eagle Alternative Capital,Not Listed
661,2021-05-26,25.570000,25.570000,25.57,25.57,0.0,0.0,0.0,FCRW,First Eagle Alternative Capital,Not Listed
662,2021-05-27,25.559999,25.559999,25.51,25.51,200.0,0.0,0.0,FCRW,First Eagle Alternative Capital,Not Listed


In [ ]:
real_df.to_csv('stock_history_data.csv')
# # Creates empty lists to save tickers, company names,
# # and stock exchange information

# tickers = []
# name = []
# exchange = []

# # Loops through stocks list and fills each empty list
# for s in stocks:
#     info = s.split("|")
#     tickers.append(info[0])
#     name.append(info[1])
#     exchange.append(info[2])

In [ ]:
# # Creates dictionary with ticker symbol, company name,
# # and stock exchange information
# info_dict = {'Ticker Symbol':tickers,'Company':name,'Exchange':exchange}

# # Creates new pandas dataframe containing dictionary information
# info_df = pd.DataFrame(info_dict)

# tickers = info_df['Ticker Symbol']
# tickers



In [ ]:
# Update: the code below has been commented out
# due to the use of a new Polygon.io subscription
# which eliminates the need for a sleep function
# New for loop posted below

In [ ]:
# # Initialize list to hold information from polygon.io
# polygon_info = []
# polygon_div = []

# startTime = time.time()
# #today = date.today()
# #todays_date = today.strftime("%Y-%m-%d")

# # Initializes counter to 0
# count = 0

# error_log = open("errors.log","a")
    
# # Loops through one list of ticker symbols
# for ticker in tickers:

#     # Checks if count equals 4, used to prevent exceeding 
#     # the API call limit of 5 calls/minute
    
#     # Update: with new subscription, no if statement needed
#     # If statement to be commented out
#     #if count == 4:

#         # Tells program to wait 1.5 minutes before running again
#         #time.sleep(60)

#         # Ticker symbols to be used for API Call
#         #dataset_code = ticker

#         # URLs for API Calls, the first for open and closed prices, 
#         # the second for divident information
#         #url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
#         #div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

#         # Saving responses from both urls
#         #try:
#         #    response = requests.get(url).json()
#         #    div_response = requests.get(div_url).json()

#             # Adding responses to info and div lists, respectively
#         #    polygon_info.append(response)
#             polygon_div.append(div_response)

#         except:
#             error_log.write(ticker)
#             print(f"On {ticker}, error trying to call")

#         # Resets counter
#         count = 0

#     else:
#         # Ticker symbols to be used for API Call
#         dataset_code = ticker

#         # URLs for API Calls, the first for open and closed prices, 
#         # the second for divident information
#         url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
#         div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

#          # Saving responses from both urls
#         try:
#             response = requests.get(url).json()
#             div_response = requests.get(div_url).json()

#             # Adding responses to info and div lists, respectively
#             polygon_info.append(response)
#             polygon_div.append(div_response)

#         except:
#             error_log.write(ticker)
#             print(f"On {ticker}, error trying to call")

#         # Increase counter by 2
#         count += 2



In [ ]:
# Initialize lists to hold information from polygon.io
# polygon_info = []
# polygon_div = []

# error_log = open("errors.log","a")
# bad_ticker_log = open("bad_ticker.log","a")

# count = 1

# # Captures start time for loop
# startTime = time.time()

# for ticker in tickers:
# # Ticker symbols to be used for API Call
    
#     if count > 5:
        
#         time.sleep(65)
        
#         dataset_code = ticker

#         # URLs for API Calls, the first for open and closed prices, 
#         # the second for divident information
#         url = f"https://api.polygon.io/v1/open-close/{dataset_code}/{todays_date}?unadjusted=true&apiKey={api_key}"
#         div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

#          # Saving responses from both urls
#         try:
#             response = requests.get(url).json()
#             div_response = requests.get(div_url).json()

#             if response['status'] == 'OK':
#             # Adding responses to info and div lists, respectively
#                 polygon_info.append(response)
#                 polygon_div.append(div_response)
                
#             else:
#                 bad_ticker_log.write(ticker)

#         except:
#             error_log.write(ticker)
#             print(f"On {ticker}, error trying to call")
            
#         count = 1
        
#     else:
        
#         dataset_code = ticker

#         # URLs for API Calls, the first for open and closed prices, 
#         # the second for divident information
#         url = f"https://api.polygon.io/v1/open-close/{dataset_code}/{todays_date}?unadjusted=true&apiKey={api_key}"
#         div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

#          # Saving responses from both urls
#         try:
#             response = requests.get(url).json()
#             div_response = requests.get(div_url).json()

#             if response['status'] == 'OK':
#             # Adding responses to info and div lists, respectively
#                 polygon_info.append(response)
#                 polygon_div.append(div_response)
                
#             else:
#                 bad_ticker_log.write(ticker)

#         except:
#             error_log.write(ticker)
#             print(f"On {ticker}, error trying to call")
            
#         count += 1
            
# executionTime = (time.time() - startTime)

In [ ]:
# realTime = executionTime/60
# realTime_hr = realTime/60
# realTime_hr

In [ ]:
# len(polygon_info)

In [ ]:
# polygon_info_2 = []
# polygon_div_2 = []

# read_error = open("errors.log","r")
# startTime = time.time()

# # Loops through one list of ticker symbols
# for ticker in read_error:
#     # Ticker symbols to be used for API Call
#     dataset_code = ticker

#     # URLs for API Calls, the first for open and closed prices, 
#     # the second for divident information
#     url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
#     div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"

#     # Saving responses from both urls
#     try:
#         response = requests.get(url).json()
#         div_response = requests.get(div_url).json()

#         # Adding responses to info and div lists, respectively
#         polygon_info_2.append(response)
#         polygon_div_2.append(div_response)

#     except:
#         print(f"On {ticker}, error trying to call")

In [ ]:
# executionTime = (time.time() - startTime)

# realTime = executionTime/60
# realTime

In [ ]:
# real_info = []

# for t in polygon_info:
#     if t['status'] == 'OK':
#         real_info.append(t)
        
# len(real_info)

In [ ]:
# real_info_df = pd.DataFrame(real_info)
# real_info_df = real_info_df[['symbol','from','open','high','low','close','volume']]
# #polygon_info_df.columns = ['symbol','date','open','high','low','close','volume']
# real_info_df

In [ ]:
# all_div = polygon_div[1:]
# all_div

# # len(all_div)

# # for div in all_div:
# #     print(div['results'])
# #     print()

In [ ]:
# real_div = []

# for t in polygon_div[1:]:
#     if t['status'] == 'OK' and t['count'] > 0:
#         real_div.append(t)

In [ ]:
# #all_div = polygon_div[1:]
# div_amt_list = []
# payDate = []
# ticker_list = []

# for div in real_div:
#     result = div['results']
    
#     if result == []:
#         continue
        
#     else:
#         ticker = div['results'][0]['ticker']
#         div_amt = div['results'][0]['amount']
#         pay_date = div["results"][0]['paymentDate']
    
#     ticker_list.append(ticker)
#     div_amt_list.append(div_amt)
#     payDate.append(pay_date)
    
# polygon_div_df = pd.DataFrame({"Symbol":ticker_list,"Amount ($)":div_amt_list,"Payment Date":payDate})

In [ ]:
# polygon_div_df

In [ ]:
# polygon_info_df.to_csv('openClose.csv')
# polygon_div_df.to_csv('dividend.csv')
# company_table.to_csv('companies.csv')

In [ ]:
# polygon_info_df = pd.read_csv('openClose.csv')
# polygon_div_df = pd.read_csv('dividend.csv')
# company_table = pd.read_csv('companies.csv')

# polygon_info_df.columns = ['index','symbol','date','opening','high','low','close','volume']
# polygon_info_df = polygon_info_df.drop(columns='index')
# polygon_div_df

In [ ]:
# del polygon_div_df['Unnamed: 0']

In [ ]:
# del company_table['Unnamed: 0']
# company_table

In [ ]:
# # Create sqlite database path
# database_path = "russell_stock.sqlite"

# # Create an engine that can talk to the database
# engine = create_engine(f"sqlite:///{database_path}")

In [ ]:
# # use engine to connect to existing tables/db
# Base = declarative_base()
# #conn = sqlite3.connect('russell_stock.sqlite')
# #c = conn.cursor()

# # Use this to clear out the db
# # ----------------------------------
# Base.metadata.drop_all(bind=engine)

# # Create a "Metadata" Layer That Abstracts our SQL Database
# # ----------------------------------
# #Base.metadata.create_all(engine)

In [ ]:
# # Define our General_Info table
# class Open_Close(Base):
#     __tablename__ = 'openClose'
#     __table_args__ = {'extend_existing': True}
#     symbol = Column(String, primary_key=True)
#     date = Column(String)
#     opening = Column(Float)
#     high = Column(Float)
#     low = Column(Float)
#     close = Column(Float)
#     volume = Column(Integer)

# # Define our Dividend table
# class Dividend(Base):
#     __tablename__ = 'dividend'
#     __table_args__ = {'extend_existing': True}
#     symbol = Column(String, primary_key=True)
#     amount = Column(Float)
#     payment_date = Column(String)

# # Define our Company Table
# class Company(Base):
#     __tablename__ = 'company'
#     __table_args__ = {'extend_existing': True}
#     symbol = Column(String, primary_key=True)
#     name = Column(String)

In [ ]:
# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
#Open_Close = Database.classes.open_close

In [ ]:
# #for i in polygon_info_df.index:
# #    print(polygon_info_df['from'][i])


# # insert dataframe into sqlite tables
# company_table.to_sql(name='company', con=engine, if_exists='replace', index=True)
# polygon_div_df.to_sql(name='dividend', con=engine, if_exists='replace', index=True)
# polygon_info_df.to_sql(name='openClose', con=engine, if_exists='replace', index=True)

In [ ]:
# # for i in polygon_info_df.index:
#     thisFrom = polygon_info_df["date"][i]
#     thisSymbol = polygon_info_df["symbol"][i]
#     thisOpen = polygon_info_df["open"][i]
#     thisHigh = polygon_info_df["high"][i]
#    thisLow = polygon_info_df["low"][i]
#    thisClose = polygon_info_df["close"][i]
#    thisVol = polygon_info_df["volume"][i]
    
    #row = [thisFrom,thisSymbol,thisOpen,thisHigh,thisLow,thisClose,thisVol]
    
#    thisCompany = Open_Close(date=thisFrom,symbol=thisSymbol,opening=thisOpen,high=thisHigh,low=thisLow,close=thisClose,volume=thisVol)
#    session = Session(bind=engine)
    #conn.execute('Insert into open_close values(?,?,?,?,?,?,?)', row)
    #engine.execute()
#    Session.configure(bind=engine)
#    session.add(thisCompany)
    #session.commit()
    
#session.close()
    
    
    
    

In [ ]:
#c.close()
#session.commit()



In [ ]:
# #session.close()

# #conn.close()

# engine.dispose()

In [ ]:
# engine = create_engine(f"sqlite:///{database_path}")

# connection = engine.connect()

# # use engine to connect to existing tables/db
# base = automap_base()
# base.prepare(engine, reflect=True)

# # View all of the classes/tables that automap found
# thing = base.classes.keys()

# print(thing)

# # Save references to each table (capital because they are considered classes) 
# #Company = Database.classes.company
# #Dividend = Database.classes.dividend
# inspector = inspect(engine)

In [ ]:
# connection.close()

# engine.dispose()

In [ ]:
import os

for i in range(0,2000,100):
    start = i
    stop =  i + 100
    script_text = '''{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Untitled0.ipynb",
      "provenance": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "32ADhwU0Vgmv",
        "outputId": "2a4e86e5-504a-4a0b-a2ff-c709da94d81e"
      },
      "source": [
'''
    script_text_2 = f'''        "!pip install yfinance",
        "!git clone https://github.com/emjasmine/TeamMoneyPenny-ML.git",
        "%cd TeamMoneyPenny-ML",
        "!pwd",
        "!python yfinance_API_client_end.py {start} {stop}",
        "!ls",
        "!git add .",
        "!git commit -m \"Adding new files from {start} to {stop}\"",
        "!git push"'''
    script_text_3 = '''
      ],
      "execution_count": 19,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "0-10_stock_info.csv  nasdaq_screener.csv  templates",
            "0-2_stock_info.csv   nasdaq_screener.xls  US-Stock-Symbols.csv",
            "0-3_stock_info.csv   nyse_list.csv\t  US-Stock-Symbols.xlsx",
            "0-5_stock_info.csv   nyse_list.xls\t  yfinance_API_client_end.py",
            "API_Calls.ipynb      README.md",
            "errors.log\t     static"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "BiTqAdc4WLCn"
      },
      "source": [
        "# New Section"
      ]
    }
  ]
}
'''
    path = os.path.join('Colabs',f'colab_{start}_{stop}.ipynb')
    file = open(path,'w')
    file.write(script_text+script_text_2+script_text_3)
    file.close()


In [ ]:
!ls